<center><img src="Fig/UGA.png" width="30%" height="30%"></center>
<center><h3>Master of Science in Industrial and Applied Mathematics (MSIAM)  -- 1st year</h3></center>
<hr>
<center><h1>Numerical Optimization</h1></center>
<center><h2>Lab 9: constrained optimization, basics</h2></center>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from plotLib import *
%matplotlib inline

---

# Constrained optimization

We consider the problem of miniizing a function $f$ under inequality constraints:

\begin{equation}
\min_{x\in\mathbb R^n} f(x) \qquad \text{s.t.}\qquad c_i(x)\le 0, \quad i=1, \ldots, m.
\label{eq:gereral_problem} \tag{1}
\end{equation}

Note that this setting also encompasses equality constraints.

### Simple feasible sets
In previous labs, we discussed the projected gradient method. It allows to tackle problems which feasible set, defined by the constraints $c_i(x)\le 0$, can be easily projected upon. These sets were mainly $\mathbb R^+$, $[a, b]$ where $a, b\in\{-\infty\}\cup\mathbb R\cup\{+\infty\}$, or products of such sets. One iteration writes

\begin{equation}
    x_{k+1} = \mathrm{proj}_{C} (x_k - \gamma \nabla f(x_k))
\end{equation}

where $C = \{x : c_i(x)\le 0, \quad i=1, \ldots, m\}$, and $\gamma>0$ is a stepsize.

### Generic feasible sets: primal and dual problem
We now turn to the more general problem $\eqref{eq:gereral_problem}$, for projection on the feasible space is a problem almost as difficult as solving the full problem. In this lab, we turn to methods stemming from the duality theory.

#### Primal problem
The so-called *primal problem* associated with $\eqref{eq:gereral_problem}$ writes:

\begin{equation}
\min_{x\in\mathbb R^n} \max_{\lambda \in\mathbb{R}_{+}^m} f(x) + \langle \lambda, c(x) \rangle
\label{eq:primal} \tag{$\mathcal P$}
\end{equation}

> **Task 1**: Compare problems $\eqref{eq:gereral_problem}$ and $\eqref{eq:primal}$. You may consider the function $p$ defined by:

\begin{equation}
p(x) = \max_{\lambda \in\mathbb{R}_{+}^m} f(x) + \langle \lambda, c(x) \rangle.
\end{equation}

#### Dual problem
One may also consider the so-called *dual problem*:

\begin{equation}
\max_{\lambda \in\mathbb{R}_{+}^m} \min_{x\in\mathbb R^n} f(x) + \langle \lambda, c(x) \rangle,
\label{eq:dual} \tag{$\mathcal D$}
\end{equation}

and define the *dual function*

\begin{equation}
q(\lambda) = \min_{x\in\mathbb R^n} f(x) + \langle \lambda, c(x) \rangle.
\end{equation}

You may recognize here the basic objects on which the duality theory builds, including the Langrangian function of $\eqref{eq:gereral_problem}$:

$$L(x, \lambda) = f(x) + \langle \lambda, c(x)\rangle .$$

Under suitable assumptions, all solutions $(\bar{x}, \bar{\lambda})$ of problem $\eqref{eq:primal}$ are solutions of problem $\eqref{eq:dual}$, and in particular, $\bar{x}$ is a solution of problem $\eqref{eq:gereral_problem}$. The problems considerd in this lab all fall into this well-behaved category.

#### Solving the dual
The projected gradient attempts to solve $\eqref{eq:primal}$ directly, which makes it a *primal method*. We consider *dual methods*, designed to solve $\eqref{eq:dual}$. In doing so, we trade the problem of minimizing an explicit function on a difficult set by that of minimizing a complex function (defined as a minimization) on a simple set.

_Fact_: Let $\lambda\in\mathbb R^m$ such that a solution to $\min_{x\in\mathbb R^n} f(x) + \langle \lambda, c(x) \rangle$ exists, which we denote $\bar{x}_\lambda$. If $\bar{x}_\lambda$ is the unique solution of this problem, then

\begin{cases} 
q(\lambda ) = f(\bar{x}_\lambda) + \langle \lambda, c(\bar{x}_\lambda) \rangle \\
\nabla q(\lambda ) = c(\bar{x}_\lambda)
\end{cases}

> **Task 2**: What algorithm could be employed to solve $\eqref{eq:dual}$?

The goal of this lab is to implement this algorithm. We'll require a stopping criterion for that algorithm, and example problems.

## A simple problem

As a first problem, we consider:

\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & 4 (x_1-3)^2 + 2 (x_2-1)^2 \\
& \text{subject to}
& & x_1 - x_2 - 1 \le 0
\end{aligned}
\end{equation*}


> **Task 3**: implement the oracles in the next cell.

In [ ]:
class SimplePb:
    def f(self, x):
        return 4*(x[0]-3)**2+2*(x[1]-1)**2

    def f_grad(self, x):
        # TODO
        return np.array([0, 0])

    def f_grad_hessian(self, x):
        # TODO
        g = np.array([0, 0])
        H = np.array([(0, 0), (0, 0)])
        return g, H

    def c1(self, x):
        return x[0] - x[1] - 1

    def c(self, x):
        return np.array([ c1(x)])

In [ ]:
simplepb = SimplePb()

x1_min = -0.5
x1_max = 5.5
x2_min = -0.5
x2_max = 2.5
levels = [0.5,1,2,5,10,15]
level_plot(simplepb.f, x1_min, x1_max, x2_min, x2_max, 200, levels , 'f: quadratic' )

c1levels = [-3,-2,-1,0,1,2,3]
level_plot( simplepb.c1, x1_min, x1_max, x2_min, x2_max, 200, c1levels , 'c_1')

> **Task 4**: Given the graphs, what is the minimizer pair $(\bar{x}, \bar{\lambda})$ for this problem.

## A stopping criterion
The algorithm coming up will generate a sequence of iterates that should get close to a solution of the initial constrained problem. We seek to formulate a criterion which determines if a given point is a solution, up to some precision. To do so, we first need to formalise the notion of solution of the constrained problem, and determine what are the mathematical properties of such points.

> **Task 5**: 
> 1. How does one formalizes the notion of "solution" of the constrained problem?
> 2. What properties do these points all verify? In other words, how does the necessary conditions of optimality write?
> 3. What parts of these relations need to be relaxed when implementing numerically?

## The gradient ascent algorithm
The *gradient ascent* algorithm, also known as *Uzawa's algorithm* consists in the following iteration:

\begin{equation*}\left|
\begin{array}{l}
    x_{k+1} = \arg\min_{x\in\mathbb R^n} f(x) + \langle \lambda^k, c(x)\rangle \\ 
    \lambda_{k+1} = \mathrm{proj}_{\mathbb{R}_-^m}\left( \lambda^k + \gamma c(x_{k+1}) \right)
\end{array}\right.
\end{equation*}

where $\gamma>0$ is some chosen stepsize.

> **Task 6**: The frist part of the iteration consists in solving an otpimization problem. What kind of problem is it? What ways to solve it can you think of? Which is most relevant?

*Note*: 
- You may find it useful for this task to peek at the problems considered next. Overall, there will be several constraints ($m>1$), but all individual constraints will be quadratic functions. How does one solve a quadratic optimization problem?
- depending on how you choose to solve this subproblem, you may want to add functions computing the lagrangian value, gradient and hessian to the classes defining the problems.

> **Task 7**: Implement the Uzawa algorithm as described above, and check it on the above toy problem.

*Note*: gradient ascent converges for any starting point, and for stepsizes $\gamma \in (0, 2\mu/\tau^2)$, where $\mu$ is a strong convexity constant for $f$ and $\tau$ a Lipschitz continuity constant for $c$.

## More involved problems

You may now experiment with your algorithm on more involved problems.

### 1. Two affine active constraints

We consider the problem

\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & 4 (x_1-4)^2 + 2 (x_2-1)^2 \\
& \text{subject to}
& & 2 x_1 - x_2 - 4 \le 0 \\
& & & x_1 - 3 \le 0
\end{aligned}
\end{equation*}

> **Task 8**: implement the oracles in the next cell.

In [ ]:
class ActiveAffinePb:
    def f(self, x):
        return 4*(x[0]-4)**2+2*(x[1]-1)**2

    def f_grad(self, x):
        # TODO
        return np.array([0, 0])

    def f_grad_hessian(self, x):
        # TODO
        g = np.array([0, 0])
        H = np.array([(0, 0), (0, 0)])
        return g, H

    def c1(self, x):
        return 2*x[0] - x[1] - 4
    
    def c2(self, x):
        return x[0] - 3

    def c(self, x):
        return np.array([ c1(x), c2(x)])

In [ ]:
activeaffpb = ActiveAffinePb()

x1_min = -0.5
x1_max = 5.5
x2_min = -0.5
x2_max = 2.5
levels = [0.5,1,2,5,10,15]
level_plot(activeaffpb.f, x1_min, x1_max, x2_min, x2_max, 200, levels , 'f: quadratic' )

c1levels = [-3,-2,-1,0]
level_plot( activeaffpb.c1, x1_min, x1_max, x2_min, x2_max, 200, c1levels , 'c_1')
c2levels = [-3,-2,-1,0]
level_plot( activeaffpb.c2, x1_min, x1_max, x2_min, x2_max, 200, c2levels , 'c_2')

> **Task 9**: Run your algorithm on the above problem. Inspect the solution pair, and propose a geometrical interpretation of the KKT conditions. 

### 2. An ellipse constraint

We consider the problem

\begin{equation*}
\begin{aligned}
& \underset{x}{\text{minimize}}
& & 4 (x_1-4)^2 + 2 (x_2-1)^2 \\
& \text{subject to}
& & x_1 - x_2 - 1 \le 0
\end{aligned}
\end{equation*}

> **Task 8**: implement the oracles in the next cell.

In [ ]:
class EllipseCstrPb:
    def f(self, x):
        return 4*(x[0]-4)**2+2*(x[1]-1)**2

    def f_grad(self, x):
        # TODO
        return np.array([0, 0])

    def f_grad_hessian(self, x):
        # TODO
        g = np.array([0, 0])
        H = np.array([(0, 0), (0, 0)])
        return g, H

    def c1(self, x):
        return 0.5 * (x[0] - 1)**2 + x[1]**2 - 1

    def c(self, x):
        return np.array([ c1(x) ])

In [ ]:
ellipsecstrpb = EllipseCstrPb()

x1_min = -0.5
x1_max = 5.5
x2_min = -0.5
x2_max = 2.5
levels = [0.5,1,2,5,10,15]
level_plot(ellipsecstrpb.f, x1_min, x1_max, x2_min, x2_max, 200, levels , 'f: quadratic' )

c1levels = [-0.5, 0, 1, 2, 4, 8]
level_plot( ellipsecstrpb.c1, x1_min, x1_max, x2_min, x2_max, 200, c1levels , 'c_1')

## Going further

The above method is known as *gradient ascent*, or the *Uzawa method*. It is one of the most elementary *dual methods* for solving constrained optimization programs. As such, it has several drawbacks:
- each iteration requires solving one optimization problem;
- its convergence garantees are quite restrictive (notably, $f$ should be strongly convex).

The first issue can be dealt with by doing only a partial optimization of the subproblem. Performing one gradient step yields the *Arrow-Hurwicz* algorithm:

\begin{equation*}\left|
\begin{array}{l}
    x_{k+1} = x_k - \epsilon \nabla_x L(x_k, \lambda_k) \\ 
    \lambda_{k+1} = \mathrm{proj}_{\mathbb{R}_-^m}\left( \lambda^k + \gamma \nabla_\lambda L(x_{k+1}, \lambda_k) \right)
\end{array}\right.
\end{equation*}
for some stepsizes $\epsilon, \gamma>0$, and $\nabla_x L(x, \lambda)$ denotes the gradient of $L(\cdot, \lambda)$ (thus relative to the $x$ vairable).

The second is more difficult to alleviate. One prolific research direction consisted in changing the structure of the langrangian (and thus the notion of duality used). One very popular algorithm of this last decade is the *Alternating Direction Method of Multiplier*, or *ADMM*.

Finally, as for unconstrained optimization, there exist methods which converge locally fast (*quadratically*) for constrained problems. The *Sequential Quadratic Programming* (*SQP*) class of algorithms is a typical with fast local convergence. Besides, one should keep in mind that for small to medium sized problems, *interior point methods* are the best methods in term of speed of convergence.

#### References:
- gradient, newton and interior point methods: _Numerical Optimization_, J. Frédéric Bonnans, J. Charles Gilbert, Claude Lemaréchal, and Claudia Sagastizabal;
- more on ADMM and dual methods: _Distributed Optimization and StatisticalLearning via the Alternating Direction Method of Multipliers_, S. Boyd _et al_;
- SQP and a nice reference for most methods seen in the labs: _Numerical Optimization_, J. Nocedal, S. Wright;